# FCOS Model

We will use the FCOS model from the [FCOS](./documentation/fcos.pdf)paper. This model don't use anchored bounding box.

## Imports

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.io import read_image
from torchvision.transforms import (
    Compose,
    Normalize,
    Resize,
    ToPILImage,
    ToTensor,
)
import torchvision

from tqdm import tqdm


## Load the Dataset

We will use a coco subset to train this model.

### Dataset Class

In [13]:
"""
Classe pour charger les données.
Les images sont dans le dossier img_dir et les labels dans un fichier csv annotations_file.
Les labels sont des entiers [0,1] qui représentent la présence d'une planche.
"""
class CrapaudDataset(Dataset):
    def __init__(
        self, annotations_file, img_dir, transform=None, target_transform=None
    ):
        self.img_labels = pd.read_json(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [14]:
# Pipeline de transformation des images
img_pipeline = Compose(
    [
        ToPILImage(),
        Resize((256, 256)),
        ToTensor(),
    ]
)

# Chargement des données
ds_train = CrapaudDataset(
    annotations_file=os.path.join("subset","coco_subset", "train.json"),
    img_dir=os.path.join("subset","coco_subset","train"),
    transform=img_pipeline,
)
ds_val = CrapaudDataset(
    annotations_file=os.path.join("subset","coco_subset", "val.json"),
    img_dir=os.path.join("subset","coco_subset","val"),
    transform=img_pipeline,
)

# Hyperparamètres
batch_size = 4
validation_split = 0.2
shuffle_dataset = True
random_seed = 42
hp = dict(num_epochs=3, learning_rate=0.001, momentum=0.9)

ValueError: All arrays must be of the same length

## Train/Test Split

In [ ]:
train_loader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(ds_val, batch_size=batch_size, shuffle=True)

In [ ]:
# function to show an image
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
batch = iter(train_loader)
images, labels = next(batch)


# show images
imshow(torchvision.utils.make_grid(images))

# print labels
print(" ".join(f"{l}" for l in labels))

RuntimeError: [Errno 2] No such file or directory: 'subset/souris'